In [ ]:
import trimesh
import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.optimize import minimize
from scipy.spatial import cKDTree

# Load meshes
mesh1 = trimesh.load('src/Sample_01_d.vtp')
mesh2 = trimesh.load('test/test.stl')

# Extract vertices
vertices1 = np.array(mesh1.vertices)
vertices2 = np.array(mesh2.vertices)

# Step 1: Align the bases
# Since the base of every mesh is flat, find the lowest Z-point in both meshes
def align_base(vertices):
    min_z = np.min(vertices[:, 2])
    vertices[:, 2] -= min_z
    return vertices

vertices1 = align_base(vertices1)
vertices2 = align_base(vertices2)

# Step 2: Scale Mesh 2 to match the size of Mesh 1
scale_factor = np.mean(np.linalg.norm(vertices1, axis=1)) / np.mean(np.linalg.norm(vertices2, axis=1))
vertices2 *= scale_factor

# Step 3: Center both meshes to the origin
center1 = np.mean(vertices1, axis=0)
center2 = np.mean(vertices2, axis=0)

vertices1 -= center1
vertices2 -= center2

# Step 4: Define an alignment function with ICP for different point counts
def align_mesh(mesh_source, mesh_target):
    target_tree = cKDTree(mesh_target)  # Create a KD-Tree for efficient nearest-neighbor search

    def objective_function(params):
        rotation = R.from_euler('xyz', params[:3]).as_matrix()
        translation = params[3:6]
        
        # Apply transformation
        transformed_vertices = (rotation @ mesh_source.T).T + translation
        # Find nearest neighbors in the target mesh
        distances, _ = target_tree.query(transformed_vertices)
        
        # Mean distance as error metric
        return distances.mean()

    # Initial rotation (0,0,0) and translation (0,0,0)
    initial_params = np.zeros(6)
    result = minimize(objective_function, initial_params, method='BFGS')
    optimal_params = result.x

    # Extract optimized rotation and translation
    rotation_optimal = R.from_euler('xyz', optimal_params[:3]).as_matrix()
    translation_optimal = optimal_params[3:6]
    
    # Transform source vertices
    transformed_vertices = (rotation_optimal @ mesh_source.T).T + translation_optimal
    return transformed_vertices

# Align vertices2 to vertices1
aligned_vertices2 = align_mesh(vertices2, vertices1)

# Create an updated mesh with the aligned vertices
aligned_mesh2 = trimesh.Trimesh(vertices=aligned_vertices2, faces=mesh2.faces)

# Save the aligned mesh2 if needed
aligned_mesh2.export('aligned_mesh2.obj')


In [8]:
import vtk


reader = vtk.vtkXMLPolyDataReader()
reader.SetFileName('src/Sample_07_d.vtp')
reader.Update()
poly_data = reader.GetOutput()

transform = vtk.vtkTransform()
transform.Scale(-1, -1, -1)  

transform_filter = vtk.vtkTransformPolyDataFilter()
transform_filter.SetInputData(poly_data)
transform_filter.SetTransform(transform)
transform_filter.Update()
flipped_poly_data = transform_filter.GetOutput()

writer = vtk.vtkXMLPolyDataWriter()
writer.SetFileName('src/Sample_01007_d.vtp')
writer.SetInputData(flipped_poly_data)
writer.Write()

1